In [ ]:
# تثبيت المكتبات المطلوبة
# !pip install pose-format mediapipe opencv-python simple-video-utils tqdm

## 1️⃣ استخراج Landmarks من فيديو واحد

### الطريقة الأولى: استخدام سطر الأوامر (CLI)

In [ ]:
# استخدام CLI لتحويل فيديو إلى ملف .pose
# غيّر المسار حسب ملفك
!video_to_pose --format mediapipe -i "path/to/your/video.mp4" -o "output.pose"

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\micrk\miniconda3\envs\ML_sign_language\Scripts\video_to_pose.exe\__main__.py", line 6, in <module>
  File "C:\Users\micrk\miniconda3\envs\ML_sign_language\Lib\site-packages\pose_format\bin\pose_estimation.py", line 82, in main
    raise FileNotFoundError(f"Video file {args.i} not found")
FileNotFoundError: Video file r./test/original_videos/ESL_161_ESL_كتب_كتاب.mp4 not found


In [ ]:
# مع إعدادات إضافية لدقة أعلى
!video_to_pose --format mediapipe -i "path/to/your/video.mp4" -o "output_hq.pose" \
  --additional-config="model_complexity=2,smooth_landmarks=false,refine_face_landmarks=true"

### الطريقة الثانية: استخدام Python مباشرة

In [4]:
import os
from pose_format import Pose
from pose_format.utils.holistic import load_holistic
from simple_video_utils.metadata import video_metadata
from simple_video_utils.frames import read_frames_exact

def extract_pose_from_video(video_path: str, output_path: str, model_complexity: int = 1):
    """
    استخراج landmarks من فيديو وحفظها كملف .pose
    
    Parameters:
    -----------
    video_path : str
        مسار الفيديو المدخل
    output_path : str
        مسار ملف الـ pose الناتج
    model_complexity : int
        تعقيد النموذج (0, 1, أو 2). 2 = أعلى دقة لكن أبطأ
    """
    print(f"🎬 جاري تحميل الفيديو: {video_path}")
    
    # قراءة معلومات الفيديو
    metadata = video_metadata(video_path)
    width = metadata.width
    height = metadata.height
    fps = metadata.fps
    
    print(f"📐 الأبعاد: {width}x{height}")
    print(f"🎞️ معدل الإطارات: {fps} FPS")
    
    # قراءة إطارات الفيديو
    frames = read_frames_exact(video_path)
    
    # إعدادات MediaPipe Holistic
    holistic_config = {
        'model_complexity': model_complexity,
        'smooth_landmarks': True,
        'refine_face_landmarks': True  # للحصول على نقاط قزحية العين
    }
    
    print("⚙️ جاري استخراج النقاط...")
    
    # استخراج الـ pose
    pose = load_holistic(
        frames,
        fps=fps,
        width=width,
        height=height,
        progress=True,  # إظهار شريط التقدم
        additional_holistic_config=holistic_config
    )
    
    # حفظ النتيجة
    print(f"💾 جاري الحفظ في: {output_path}")
    file_name = os.path.basename(video_path)
    name, _ = os.path.splitext(file_name)
    output_path = os.path.join(output_path, f"{name}.pose")
    with open(output_path, 'wb') as f:
        pose.write(f)
    
    print("✅ تم بنجاح!")
    return pose

In [8]:
# مثال على الاستخدام
# غيّر المسارات حسب ملفاتك

VIDEO_PATH = "./test/original_videos/ESL_161_ESL_كتب_كتاب.mp4"
OUTPUT_PATH = "./test/pose_files/"

# استخراج الـ pose
pose = extract_pose_from_video(VIDEO_PATH, OUTPUT_PATH, model_complexity=2)

🎬 جاري تحميل الفيديو: ./test/original_videos/ESL_161_ESL_كتب_كتاب.mp4
📐 الأبعاد: 1920x1080
🎞️ معدل الإطارات: 30.0 FPS
⚙️ جاري استخراج النقاط...


52it [00:18,  2.78it/s]

💾 جاري الحفظ في: ./test/pose_files/
✅ تم بنجاح!


## 2️⃣ استخراج Landmarks من مجلد فيديوهات

### الطريقة الأولى: استخدام CLI

In [ ]:
# معالجة مجلد كامل مع عدة عمال (parallel processing)
!videos_to_poses --format mediapipe --directory "/path/to/videos" --num-workers 4 --recursive

### الطريقة الثانية: استخدام Python

In [14]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# صيغ الفيديو المدعومة
SUPPORTED_VIDEO_FORMATS = [".mp4", ".mov", ".avi", ".mkv", ".flv", ".wmv", ".webm"]

def find_videos(directory: str, recursive: bool = False):
    """
    البحث عن ملفات الفيديو في مجلد
    """
    directory = Path(directory)
    videos = []
    
    glob_method = directory.rglob if recursive else directory.glob
    
    for ext in SUPPORTED_VIDEO_FORMATS:
        videos.extend(glob_method(f"*{ext}"))
    
    return sorted(videos)

def process_single_video(video_path: Path, output_dir: Path, model_complexity: int = 1):
    """
    معالجة فيديو واحد وحفظ النتيجة في المجلد المحدد
    """
    try:
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        output_path = output_dir / f"{video_path.stem}.pose"
        
        # تخطي إذا كان الملف موجود
        if output_path.exists():
            return f"⏭️ تم تخطي (موجود): {video_path.name}"
        
        # قراءة الفيديو
        metadata = video_metadata(str(video_path))
        frames = read_frames_exact(str(video_path))
        
        # استخراج الـ pose
        pose = load_holistic(
            frames,
            fps=metadata.fps,
            width=metadata.width,
            height=metadata.height,
            progress=False,
            additional_holistic_config={'model_complexity': model_complexity}
        )
        
        # حفظ
        with open(output_path, 'wb') as f:
            pose.write(f)
        
        return f"✅ تم: {video_path.name}"
    
    except Exception as e:
        return f"❌ فشل {video_path.name}: {str(e)}"

def batch_process_videos(directory: str, output_dir: str = OUTPUT_PATH, recursive: bool = False, 
                        num_workers: int = 1, model_complexity: int = 1):
    """
    معالجة مجلد كامل من الفيديوهات بالتوازي مع السماح بتحديد مجلد الإخراج
    يستخدم ThreadPoolExecutor بدلاً من ProcessPoolExecutor لتجنب مشاكل pickle في Jupyter/Windows
    """
    videos = find_videos(directory, recursive)
    print(f"📁 تم العثور على {len(videos)} فيديو")
    
    if not videos:
        print("⚠️ لم يتم العثور على فيديوهات!")
        return
    
    results = []
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # معالجة متوازية باستخدام Threads لتفادي مشاكل ProcessPool في Jupyter
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = {
            executor.submit(process_single_video, video, output_dir, model_complexity): video 
            for video in videos
        }
        
        for future in tqdm(as_completed(futures), total=len(videos), desc="المعالجة"):
            result = future.result()
            results.append(result)
            print(result)
    
    # إحصائيات
    success = sum(1 for r in results if r.startswith("✅"))
    skipped = sum(1 for r in results if r.startswith("⏭️"))
    failed = sum(1 for r in results if r.startswith("❌"))
    
    print(f"\n📊 الإحصائيات:")
    print(f"   ✅ نجح: {success}")
    print(f"   ⏭️ تم تخطيه: {skipped}")
    print(f"   ❌ فشل: {failed}")


In [15]:
# مثال على معالجة مجلد
# غيّر المسار حسب مجلدك

VIDEOS_DIRECTORY = "./test/original_videos/"
OUTPUT_PATH = "./test/pose_files/"

batch_process_videos(
    directory=VIDEOS_DIRECTORY,
    output_dir=OUTPUT_PATH,
    recursive=True,      # البحث في المجلدات الفرعية
    num_workers=1,       # عدد العمال المتوازيين
    model_complexity=2  # دقة النموذج
)

📁 تم العثور على 6 فيديو


المعالجة:  17%|█▋        | 1/6 [00:16<01:20, 16.13s/it]

✅ تم: ESL_143_ESL_حمل.mp4


المعالجة:  33%|███▎      | 2/6 [00:29<00:58, 14.53s/it]

✅ تم: ESL_151_ESL_حب.mp4


...

✅ تم: ESL_155_ESL_زفاف.mp4


المعالجة:  67%|██████▋   | 4/6 [00:57<00:28, 14.25s/it]

✅ تم: ESL_156_ESL_طلاق.mp4
⏭️ تم تخطي (موجود): ESL_161_ESL_كتب_كتاب.mp4


المعالجة: 100%|██████████| 6/6 [01:11<00:00, 11.88s/it]

✅ تم: ESL_173_ESL_قريب.mp4

📊 الإحصائيات:
   ✅ نجح: 5
   ⏭️ تم تخطيه: 1
   ❌ فشل: 0


## 3️⃣ فهم بنية البيانات المستخرجة

### MediaPipe Holistic يستخرج:

In [16]:
# لنفهم البيانات المستخرجة
from pose_format import Pose

def analyze_pose_file(pose_path: str):
    """
    تحليل ملف pose وعرض معلوماته
    """
    with open(pose_path, 'rb') as f:
        pose = Pose.read(f.read())
    
    print("=" * 60)
    print("📊 معلومات ملف الـ Pose")
    print("=" * 60)
    
    # معلومات Header
    print(f"\n📐 الأبعاد:")
    print(f"   العرض: {pose.header.dimensions.width}")
    print(f"   الارتفاع: {pose.header.dimensions.height}")
    print(f"   العمق: {pose.header.dimensions.depth}")
    
    # معلومات Body
    print(f"\n🎞️ معلومات البيانات:")
    print(f"   معدل الإطارات: {pose.body.fps} FPS")
    print(f"   شكل البيانات: {pose.body.data.shape}")
    print(f"   (الإطارات, الأشخاص, النقاط, الأبعاد)")
    
    frames, people, points, dims = pose.body.data.shape
    print(f"\n   📹 عدد الإطارات: {frames}")
    print(f"   👥 عدد الأشخاص: {people}")
    print(f"   📍 عدد النقاط الكلي: {points}")
    print(f"   📏 عدد الأبعاد: {dims} (X, Y, Z)")
    
    # المكونات
    print(f"\n🧩 المكونات (Components):")
    total_points = 0
    for i, comp in enumerate(pose.header.components):
        print(f"\n   {i+1}. {comp.name}")
        print(f"      النقاط: {len(comp.points)}")
        print(f"      الوصلات: {len(comp.limbs)}")
        print(f"      التنسيق: {comp.format}")
        total_points += len(comp.points)
    
    print(f"\n   📍 المجموع الكلي للنقاط: {total_points}")
    
    return pose

In [18]:
# تحليل ملف pose
# غيّر المسار
FILE_PATH = "./test/pose_files/ESL_161_ESL_كتب_كتاب.pose"

pose = analyze_pose_file(FILE_PATH)

📊 معلومات ملف الـ Pose

📐 الأبعاد:
   العرض: 1920
   الارتفاع: 1080
   العمق: 0

🎞️ معلومات البيانات:
   معدل الإطارات: 30.0 FPS
   شكل البيانات: (52, 1, 586, 3)
   (الإطارات, الأشخاص, النقاط, الأبعاد)

   📹 عدد الإطارات: 52
   👥 عدد الأشخاص: 1
   📍 عدد النقاط الكلي: 586
   📏 عدد الأبعاد: 3 (X, Y, Z)

🧩 المكونات (Components):

   1. POSE_LANDMARKS
      النقاط: 33
      الوصلات: 35
      التنسيق: XYZC

   2. FACE_LANDMARKS
      النقاط: 478
      الوصلات: 2564
      التنسيق: XYZC

   3. LEFT_HAND_LANDMARKS
      النقاط: 21
      الوصلات: 21
      التنسيق: XYZC

   4. RIGHT_HAND_LANDMARKS
      النقاط: 21
      الوصلات: 21
      التنسيق: XYZC

   5. POSE_WORLD_LANDMARKS
      النقاط: 33
      الوصلات: 35
      التنسيق: XYZC

   📍 المجموع الكلي للنقاط: 586


## 4️⃣ الوصول إلى نقاط محددة

In [19]:
import numpy as np

def get_specific_landmarks(pose: Pose):
    """
    مثال على الوصول لنقاط محددة
    """
    # البيانات الخام
    data = pose.body.data          # الإحداثيات (frames, people, points, dims)
    confidence = pose.body.confidence  # درجة الثقة (frames, people, points)
    
    # الحصول على مكون معين
    # مثال: الحصول على نقاط الجسم فقط
    body_pose = pose.get_components(["POSE_LANDMARKS"])
    print(f"نقاط الجسم فقط: {body_pose.body.data.shape}")
    
    # مثال: الحصول على اليدين فقط
    hands_pose = pose.get_components(["LEFT_HAND_LANDMARKS", "RIGHT_HAND_LANDMARKS"])
    print(f"نقاط اليدين: {hands_pose.body.data.shape}")
    
    # الوصول لنقطة محددة عبر الاسم
    # مثال: الحصول على index نقطة الأنف
    nose_index = pose.header.get_point_index("POSE_LANDMARKS", "NOSE")
    print(f"\nIndex نقطة الأنف: {nose_index}")
    
    # إحداثيات الأنف في كل الإطارات
    nose_coords = data[:, 0, nose_index, :]  # (frames, 3)
    print(f"شكل إحداثيات الأنف: {nose_coords.shape}")
    
    # مثال: الحصول على نقطة الرسغ اليمنى
    right_wrist_index = pose.header.get_point_index("POSE_LANDMARKS", "RIGHT_WRIST")
    right_wrist_coords = data[:, 0, right_wrist_index, :]
    print(f"\nإحداثيات الرسغ اليمنى في أول 5 إطارات:")
    print(right_wrist_coords[:5])
    
    return data, confidence

In [20]:
# مثال على الاستخدام
data, confidence = get_specific_landmarks(pose)

نقاط الجسم فقط: (52, 1, 33, 3)
نقاط اليدين: (52, 1, 42, 3)

Index نقطة الأنف: 0
شكل إحداثيات الأنف: (52, 3)

إحداثيات الرسغ اليمنى في أول 5 إطارات:
[[698.3128662109375 669.3767700195312 -0.8100500702857971]
 [696.1565551757812 668.143310546875 -0.741894006729126]
 [697.3593139648438 667.9427490234375 -0.757840096950531]
 [705.490234375 667.6714477539062 -0.7361369729042053]
 [730.9982299804688 669.7801513671875 -0.6853887438774109]]


## 5️⃣ قائمة نقاط MediaPipe Holistic

In [21]:
# نقاط الجسم (33 نقطة)
BODY_POINTS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER',
    'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
    'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW',
    'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY',
    'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB',
    'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE',
    'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL',
    'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

# نقاط اليد (21 نقطة لكل يد)
HAND_POINTS = [
    'WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP',
    'INDEX_FINGER_MCP', 'INDEX_FINGER_PIP', 'INDEX_FINGER_DIP', 'INDEX_FINGER_TIP',
    'MIDDLE_FINGER_MCP', 'MIDDLE_FINGER_PIP', 'MIDDLE_FINGER_DIP', 'MIDDLE_FINGER_TIP',
    'RING_FINGER_MCP', 'RING_FINGER_PIP', 'RING_FINGER_DIP', 'RING_FINGER_TIP',
    'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP'
]

# نقاط الوجه (468 أو 478 نقطة)
print(f"نقاط الجسم: {len(BODY_POINTS)}")
print(f"نقاط اليد الواحدة: {len(HAND_POINTS)}")
print(f"نقاط الوجه: 468 (أو 478 مع قزحية العين)")
print(f"\nالمجموع الكلي: 33 + 468 + 21 + 21 + 33 = 576 (بدون قزحية)")

نقاط الجسم: 33
نقاط اليد الواحدة: 21
نقاط الوجه: 468 (أو 478 مع قزحية العين)

المجموع الكلي: 33 + 468 + 21 + 21 + 33 = 576 (بدون قزحية)


## 📚 ملخص

تعلمنا في هذا الـ Notebook:

1. **استخراج landmarks من فيديو واحد** باستخدام CLI أو Python
2. **معالجة مجلد فيديوهات بالتوازي** لتسريع العملية
3. **فهم بنية البيانات** في ملفات `.pose`
4. **الوصول لنقاط محددة** مثل اليدين أو الوجه

### الخطوة التالية
انتقل إلى الـ Notebook التالي لتعلم كيفية **تحويل ملفات .pose** إلى صيغ أخرى!